In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
%cd '/content/drive/MyDrive/Colab Notebooks/datasets'
%ls

/content/drive/MyDrive/Colab Notebooks/datasets
megamillions.csv  powerball.csv


In [4]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [5]:

df = pd.read_csv("megamillions.csv")
df = df.iloc[:, 4:]
df = df.iloc[:, :-1]
number_of_features = df.shape[1]
df1 = df.copy()
print(list(df.columns))


['12.1', '44', '15', '18', '1', '42']


In [6]:
print(df.shape)
print(list(df1.columns))

(2033, 6)
['12.1', '44', '15', '18', '1', '42']


In [7]:
window_length = 7
number_of_features = df.shape[1]
#df.drop(['Draw Date'], axis=1, inplace=True)

In [8]:
train = df.copy()
train.head((window_length+1))

,12.1,44,15,18,1,42
0,14,15,48,4,24,41
1,16,32,46,9,45,26
2,47,16,31,24,46,47
3,5,10,39,17,35,38
4,2,49,13,21,22,52
5,29,20,10,17,1,36
6,46,21,12,32,18,49
7,8,7,36,48,44,9


In [9]:
train_rows = train.values.shape[0]
train_samples = np.empty([ train_rows - window_length, window_length, number_of_features], dtype=float)
train_labels = np.empty([ train_rows - window_length, number_of_features], dtype=float)
for i in range(0, train_rows-window_length):
    train_samples[i] = train.iloc[i : i+window_length, 0 : number_of_features]
    train_labels[i] = train.iloc[i+window_length : i+window_length+1, 0 : number_of_features]

In [10]:
train_samples[0]


array([[14., 15., 48.,  4., 24., 41.],
       [16., 32., 46.,  9., 45., 26.],
       [47., 16., 31., 24., 46., 47.],
       [ 5., 10., 39., 17., 35., 38.],
       [ 2., 49., 13., 21., 22., 52.],
       [29., 20., 10., 17.,  1., 36.],
       [46., 21., 12., 32., 18., 49.]])

In [11]:
train_labels[0]

array([ 8.,  7., 36., 48., 44.,  9.])

In [12]:
scaler = StandardScaler()
transformed_dataset = scaler.fit_transform(train.values)
scaled_train_samples = pd.DataFrame(data=transformed_dataset, index=train.index)

In [13]:
scaled_train_samples.head(window_length+1)

,0,1,2,3,4,5
0,-0.943925,-0.899323,0.863649,-1.459855,-0.434188,1.861042
1,-0.837618,0.004051,0.757760,-1.193687,0.680404,0.662924
2,0.810139,-0.846183,-0.036407,-0.395181,0.733480,2.340290
3,-1.422306,-1.165022,0.387149,-0.767817,0.149646,1.621419
4,-1.581766,0.907426,-0.989409,-0.554882,-0.540340,2.739663
5,-0.146623,-0.633625,-1.148242,-0.767817,-1.654932,1.461670
6,0.756986,-0.580485,-1.042353,0.030689,-0.752643,2.500039
7,-1.262845,-1.324441,0.228315,0.882428,0.627329,-0.694944


In [14]:
x_train = np.empty([ train_rows - window_length, window_length, number_of_features], dtype=float)
y_train = np.empty([ train_rows - window_length, number_of_features], dtype=float)

for i in range(0, train_rows-window_length):
    x_train[i] = scaled_train_samples.iloc[i : i+window_length, 0 : number_of_features]
    y_train[i] = scaled_train_samples.iloc[i+window_length : i+window_length+1, 0 : number_of_features]

In [15]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import mse

In [16]:
# Initialising the RNN
model = Sequential()
# Adding the input layer and the LSTM layer
model.add(Bidirectional(LSTM(240,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a first Dropout layer
model.add(Dropout(0.2))
# Adding a second LSTM layer
model.add(Bidirectional(LSTM(240,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a second Dropout layer
model.add(Dropout(0.2))
# Adding a third LSTM layer
model.add(Bidirectional(LSTM(240,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a fourth LSTM layer
model.add(Bidirectional(LSTM(240,
                        input_shape = (window_length, number_of_features),
                        return_sequences = False)))
# Adding a third Dropout layer
model.add(Dropout(0.2))
# Adding the first output layer
model.add(Dense(70))
# Adding the last output layer
model.add(Dense(number_of_features))

In [17]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import mse

In [18]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss ='mse', metrics=['accuracy'])
pd.options.mode.chained_assignment = None

In [19]:
model.fit(x=x_train, y=y_train, batch_size=100, epochs=400, verbose=2)

Epoch 1/400
21/21 - 22s - loss: 0.9607 - accuracy: 0.2947 - 22s/epoch - 1s/step
Epoch 2/400
21/21 - 0s - loss: 0.9313 - accuracy: 0.3095 - 282ms/epoch - 13ms/step
Epoch 3/400
21/21 - 0s - loss: 0.9301 - accuracy: 0.3105 - 277ms/epoch - 13ms/step
Epoch 4/400
21/21 - 0s - loss: 0.9283 - accuracy: 0.3184 - 271ms/epoch - 13ms/step
Epoch 5/400
21/21 - 0s - loss: 0.9285 - accuracy: 0.3105 - 277ms/epoch - 13ms/step
Epoch 6/400
21/21 - 0s - loss: 0.9270 - accuracy: 0.3169 - 277ms/epoch - 13ms/step
Epoch 7/400
21/21 - 0s - loss: 0.9273 - accuracy: 0.3149 - 279ms/epoch - 13ms/step
Epoch 8/400
21/21 - 0s - loss: 0.9268 - accuracy: 0.3228 - 269ms/epoch - 13ms/step
Epoch 9/400
21/21 - 0s - loss: 0.9260 - accuracy: 0.3253 - 288ms/epoch - 14ms/step
Epoch 10/400
21/21 - 0s - loss: 0.9248 - accuracy: 0.3169 - 272ms/epoch - 13ms/step
Epoch 11/400
21/21 - 0s - loss: 0.9248 - accuracy: 0.3184 - 287ms/epoch - 14ms/step
Epoch 12/400
21/21 - 0s - loss: 0.9251 - accuracy: 0.3233 - 297ms/epoch - 14ms/step
Epoc

In [20]:
next_Date = '2/06/2023'
print('-' * 40)
print('Predict the Future Drawing on June 02, 2023')
next = df.copy()
next = next.tail((window_length))
next = np.array(next)
x_next = scaler.transform(next)
y_next_pred = model.predict(np.array([x_next]))
print('Drawing  Date', next_Date)
print('Prediction without rounding up or down:\t', scaler.inverse_transform(y_next_pred).astype(int)[0])
print('Prediction with rounding up           :\t', scaler.inverse_transform(y_next_pred).astype(int)[0]+1)
print('Prediction with rounding down         :\t', scaler.inverse_transform(y_next_pred).astype(int)[0]-1)
print('-' * 40)

----------------------------------------
Predict the Future Drawing on June 02, 2023
1/1 [==============================] - 3s 3s/step
Drawing  Date 2/06/2023
Prediction without rounding up or down:	 [12 23 38 48 19 11]
Prediction with rounding up           :	 [13 24 39 49 20 12]
Prediction with rounding down         :	 [11 22 37 47 18 10]
----------------------------------------
